## 2 Layer MLP
- With hidden layer of size 64 and dropout with 0.5 probability
- Softmax at last classification layer 
- Learning rate of 0.001
- Over 50-100 epochs

In [1]:
import numpy as np
import tensorflow as tf
import argparse
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, root_mean_squared_error, mean_absolute_error
import math

encoder = load_model("../Models/ae_encoder_mixed.keras")

2025-09-10 21:18:36.592609: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-10 21:18:50.471761: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2025-09-10 21:18:50.531834: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 1349980000 exceeds 10% of free system memory.
2025-09-10 21:18:50.890292: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 1349980000 exceeds 10% of free system memory.
2025-09-10 21:18:51.125016: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 1349980000 exceeds 10% of free system memory.
2025-09-10 21:18:55.350415: W external/local_x

In [5]:
encoder.trainable = False
encoder.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 5000)           │   337,500,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1000)           │     5,001,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 500)            │       500,500 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 343,001,500 (1.28 GB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 343,001,500 (1.28 GB)

In [ ]:
class MLPClassifier(tf.keras.Model):
    def __init__(self, encoder, num_classes=2, hidden_size=64, dropout_rate=0.5):
        super().__init__()
        self.encoder = encoder
        self.encoder.trainable = False  # Freeze encoder
        self.hidden = layers.Dense(hidden_size, activation='relu')
        self.dropout = layers.Dropout(dropout_rate)
        self.classifier = layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.encoder(inputs)
        x = self.hidden(x)
        x = self.dropout(x)
        return self.classifier(x)

In [ ]:
model = MLPClassifier(encoder)

early_stopping = EarlyStopping(
    monitor='accuracy',
    patience=3,
    restore_best_weights=True
)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
              callbacks=[early_stopping],
              verbose=1)

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)